    Copyright 2021 Google LLC

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        https://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.


# Dialogflow CX Bot Language Translation (MAIN)

## Pre-requsite: YOU have read the README.md
1. Initial setup and initializations
2. Run the sub-notebooks as required with editable cells to modify the standard flow

The standard flow:
1. Setup blank Google Sheets
2. Query CX Agent for existing configuration (Default and Supported Languages)
3. Scan and Write CX Agent components that contains text into respective tabs in Google Sheets:
   - Intents, 
   - Entity Types
   - Flows
   - Pages
   - Route Groups
4. Cloud Translate all texts from Default Language to Target (Supported) Language/s as defined in the CX Agent
5. Apply translated text back into CX Agent

# PROVIDE Required Information in the cell below:

In [ ]:
# (MANDATORY) Provide the full URL to target Sheets and CX Agent
Google_Sheets_URL = 'https://docs.google.com/spreadsheets/.........'
CX_Agent_URL = 'https://dialogflow.cloud.google.com/cx/projects/.........'

# To access Google Sheets and Dialogflow CX, authorization is required.
# If you have NOT setup Google Sheets OAuth on this system ("credentials.json" and/or "token.json"), 
#  you have to explicitly specify a Sheets Service Account Key JSON file
#
# If you have NOT setup Google Cloud SDK OAuth to the CX Agent, you will need to specify a Dialogflow CX Service Account Key JSON file
# If filename strings are empty, we assume the Google Cloud SDK OAuth & Google Sheets OAuth tokens are already setup

GSheets_JSON  = '' #path/filename to JSON (optional)
CX_Agent_JSON = '' #path/filename to JSON (optional)

# (MANDATORY) GCP Project name with Cloud Translate API enabled
Cloud_Translate_Project_ID = 'my-cloud-translation-gcp-project.........' 


### ipynb env:

In [ ]:
!python3 -V
#!python3 -m pip list | wc -l
#!python3 -m pip list | grep google

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Initial Setup, Verification & Validation

### Imports

In [ ]:
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account

import google.cloud.dialogflowcx_v3beta1.services as cx_services
import google.cloud.dialogflowcx_v3beta1.types as cx_types

import pandas as pd
import html
import json
import copy
import time
import bs4
from pprint import pprint

from google.cloud.translate_v3beta1.services.translation_service import TranslationServiceClient
from google.cloud.translate_v3beta1.types.translation_service import TranslateTextRequest, TranslateTextResponse, Translation
Cloud_Translate_Parent = f'projects/{Cloud_Translate_Project_ID}'
translate_client = TranslationServiceClient()

### Verification

In [ ]:
# Returns a list of all JSON (*.json) files in current directory 
def get_json_files():
    json_files = []
    for file in os.listdir('./'):
        if file.endswith('.json'):
            json_files.append(file)
    return json_files

def get_cx_id(agent_uri, before_str, after_str):
    #strip from "/" + after_str
    index = agent_uri.find('/'+after_str)
    cx_id = agent_uri[:index]
    #strip up to before_str + "/"
    index = cx_id.find(before_str+'/') + len(before_str+'/')
    cx_id = cx_id[index:]
    #print(f"def get_cx_id: cx_id=[{cx_id}]")
    return cx_id

In [ ]:
#################################
## Google Sheets and CX Agent URL
#################################

if len(Google_Sheets_URL) != 0 and len(CX_Agent_URL) != 0:
    if not Google_Sheets_URL.startswith('https://docs.google.com/spreadsheets'):
        raise Exception(f'Google_Sheets_URL: [{Google_Sheets_URL}] does not look right.  Please correct it before running this Notebook.')
     
    keywords = ['https://', 'dialogflow', 'cx/projects', 'locations', 'agents']
    if not all(keyword in CX_Agent_URL for keyword in keywords):
        raise Exception(f'CX_Agent_URL: [{CX_Agent_URL}] does not look right. The keywords {keywords} were not found.  Please correct it before running this Notebook.')
        
else:
    raise Exception('Please review and provide the URLs for Google_Sheets_URL or CX_Agent_URL (or both) before running this Notebook')

##############
## OAuth JSONs
##############

# if both file names are provided, verify existence of file (if not, raise exception)
# if GSheets JSON (provided or not), check existence
# if CXAgent JSON (provide), check existence
print(f'GSheets_JSON file: "{GSheets_JSON}"\nCX_Agent_JSON file: "{CX_Agent_JSON}"\n')
print(f'INFO: JSON files found in directory: {get_json_files()}')

if len(GSheets_JSON) != 0:
    if not (os.path.isfile(GSheets_JSON)):
        raise Exception(f'Exception: GSheets_JSON file "{GSheets_JSON}" does NOT exist!')
else:
    if not ( os.path.isfile('credentials.json') or os.path.isfile('token.json') ):
        raise Exception(f'Exception: GSheets_JSON file is not specified BUT [credentials.json] and [token.json] files are NOT found!')

if len(CX_Agent_JSON) != 0:
    if not (os.path.isfile(CX_Agent_JSON)):
        raise Exception(f'Exception: CX_Agent_JSON file "{CX_Agent_JSON}" does NOT exist!')
else:
    print('CX_Agent_JSON file is not specified and we assume Google Cloud SDK OAuth has been setup on this system.')


### Global Variables & Functions

In [ ]:
Google_Sheets_ID = Google_Sheets_URL[Google_Sheets_URL.find('spreadsheets/d/')+len('spreadsheets/d/'):]
Google_Sheets_ID = Google_Sheets_ID[:Google_Sheets_ID.find('/')]
print(f'Google_Sheets_URL: [{Google_Sheets_URL}]')
print(f'Google_Sheets_ID:  [{Google_Sheets_ID}]')

CX_Agent_Link = CX_Agent_URL[CX_Agent_URL.find('projects'):]
print(f'CX_Agent_URL:  [{CX_Agent_URL}]')
print(f'CX_Agent_Link: [{CX_Agent_Link}]')

CX_Project_ID  = get_cx_id(CX_Agent_Link,'projects','locations')
CX_Location_ID = get_cx_id(CX_Agent_Link,'locations','agents')
print(f'CX_Project_ID:  [{CX_Project_ID}]')
print(f'CX_Location_ID: [{CX_Location_ID}]')

GSheets_Creds  = None
GSheets_Scopes = ['https://www.googleapis.com/auth/spreadsheets']

CX_Creds = None
CX_Client_Options = None

CX_Agent_Default_Lang = None
CX_Agent_Supported_Langs = []
Translate_Source_Lang = None

__DEBUG = False
__INFO  = True

In [ ]:
def get_cx_credentials():
    global CX_Creds
    global CX_Agent_JSON
    if len(CX_Agent_JSON) == 0:
        return None
    elif CX_Creds is None:
        CX_Creds = service_account.Credentials.from_service_account_file(CX_Agent_JSON)
    return CX_Creds

def get_cx_client_options():
    global CX_Client_Options
    if CX_Client_Options is None:
        if CX_Location_ID != 'global':
            # Reference: https://cloud.google.com/dialogflow/cx/docs/concept/region#api
            api_endpoint= f'{CX_Location_ID}-dialogflow.googleapis.com:443'
        else:
            api_endpoint= 'dialogflow.googleapis.com:443'
        CX_Client_Options = {'api_endpoint': api_endpoint}
    return CX_Client_Options
    
def get_agent():
    agents_client = cx_services.agents.AgentsClient(credentials=get_cx_credentials(), client_options=get_cx_client_options())
    
    agent_request = cx_types.GetAgentRequest()
    agent_request.name = CX_Agent_Link
    response = agents_client.get_agent(agent_request)
    
    return response

In [ ]:
def get_cx_default_lang():
    global CX_Agent_Default_Lang
    if CX_Agent_Default_Lang == None:
        agent = get_agent()
        CX_Agent_Default_Lang = agent.default_language_code
    return CX_Agent_Default_Lang
    
def get_cx_supported_langs():
    global CX_Agent_Supported_Langs
    if CX_Agent_Supported_Langs == []:
        agent = get_agent()
        CX_Agent_Supported_Langs = sorted(agent.supported_language_codes)
    return CX_Agent_Supported_Langs

def get_default_lang_for_translate():
    global Translate_Source_Lang
    if Translate_Source_Lang == None:
        # get_cx_default_lang() and take letters before '-' (if any) for Translate API call
        Translate_Source_Lang = get_cx_default_lang()
        index = Translate_Source_Lang.find('-')
        if index != -1:
            Translate_Source_Lang = Translate_Source_Lang[:index]
    return Translate_Source_Lang    
    

## Validation

In [ ]:
##############################################
### Validation - Check if we can access Sheets
##############################################

if len(GSheets_JSON) != 0:
    GSheets_Creds = service_account.Credentials.from_service_account_file(GSheets_JSON)
else:
    if os.path.exists('token.json'):
        GSheets_Creds = Credentials.from_authorized_user_file('token.json', GSheets_Scopes)
    # If there are no (valid) credentials available, let the user log in.
    if not GSheets_Creds or not GSheets_Creds.valid:
        if GSheets_Creds and GSheets_Creds.expired and GSheets_Creds.refresh_token:
            GSheets_Creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', GSheets_Scopes)
            GSheets_Creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(GSheets_Creds.to_json())
            
try:
    service = build('sheets', 'v4', credentials=GSheets_Creds)
    sheets  = service.spreadsheets().get(spreadsheetId=Google_Sheets_ID).execute()
    print(f'Validating Access to Sheets: \nSUCCESS in accessing Google Sheets at ID:[{Google_Sheets_ID}] and URL:[{Google_Sheets_URL}].\n')
except Exception as e:
    raise Exception(f'Validating Access to Sheets: \nFAILED to access Google Sheets at ID:[{Google_Sheets_ID}] and URL:[{Google_Sheets_URL}]. See below:\n{e}\n')
    
################################################
### Validation - Check if we can access CX Agent
################################################
try:
    get_agent()
    print(f'Validating Access to CX Agent: \nSUCCESS in accessing CX Agent at LINK:[{CX_Agent_Link}] and URL:[{CX_Agent_URL}].\n')
except Exception as e:
    raise Exception(f'Validating Access to CX Agent: \nFAILED to access CX Agent at LINK:[{CX_Agent_Link}] and URL:[{CX_Agent_URL}]. See below:\n{e}\n')


# 1. Run / Execute below: 


## Run the [Sheets] Notebook and Setup Sheets
### Select the cell below, from the Menu -> Cell -> "Run All Above"
Only continue to run the cells below if all the cells above ran without errors or exceptions.

In [ ]:
%run CX-Bot-Translate_Sheets.ipynb

In [ ]:
## initializes the Sheet (if empty)
## otherwise, do nothing
init_format_sheets()


## Run the [Agent] Notebook and Update Sheets

In [ ]:
%run CX-Bot-Translate_Agent.ipynb

In [ ]:
try:
    write_intents_to_sheets()
    write_entities_to_sheets()
    write_flows_to_sheets()
    write_pages_to_sheets() 
    write_route_groups_to_sheets()
except Exception as e:
    print(f'Exception:\n{e}')

## Run the [Translation] Notebook and Translate Agent in Sheets

In [ ]:
%run CX-Bot-Translate_Translation.ipynb

In [ ]:
## Feel free to comment out CX components that does not need translation
try:
    translate_cx(SheetsName.Training_Phrases)
    translate_cx(SheetsName.Entities)
    translate_cx(SheetsName.Flows)
    translate_cx(SheetsName.Pages)
    translate_cx(SheetsName.Route_Groups)
except Exception as e:
    print(f'Exception:\n{e}')

## Update Translations in Sheets to Agent
Run the cells below when you are satisfied with translations in Sheets and you are ready to apply it to Dialogflow CX

In [ ]:
## Feel free to comment out CX components that does not require updates from Sheets back to CX
try:
    update_all_intents()
#     update_intents_by_lang('Chinese - Simplified')
    update_all_entities()
#     update_entities_by_lang('Tamil')
    update_all_flows()
#     update_flows_by_lang('Tamil')
    update_all_pages()
#     update_pages_by_lang('Chinese - Simplified')
    update_all_route_groups()
#     update_route_groups_by_lang('Tamil')
except Exception as e:
    print(f'Exception:\n{e}')

# END